In [2]:
! pip install --quiet git-remote-codecommit


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import boto3
import sagemaker
import time
from time import strftime

boto_session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=boto_session)
sm_client = boto3.client("sagemaker")
region = boto_session.region_name
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
account = sagemaker_session.boto_session.client("sts").get_caller_identity()["Account"]

prefix = 'sagemaker-intel-dvc'

# print(f"account: {account}")
# print(f"bucket: {bucket}")
# print(f"region: {region}")
# print(f"role: {role}")

In [4]:
from sagemaker.pytorch.processing import PyTorchProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role

In [5]:
dvc_repo_url = "codecommit::us-west-2://sagemaker-intel"
dvc_branch = "processed-dataset"

In [12]:
pytorch_processor = PyTorchProcessor(
    framework_version='1.12',
    image_uri='****.dkr.ecr.us-west-2.amazonaws.com/emlo:infer',
    role=get_execution_role(),
    instance_type='ml.t3.medium',
    instance_count=1,
    base_job_name='preprocess-intel-dataset',
    env={
        "DVC_REPO_URL": dvc_repo_url,
        "DVC_BRANCH": dvc_branch,
        "GIT_USER": "m",
        "GIT_EMAIL": "m@emlo.com"
    }
)

In [13]:
! pwd

/root/intel-notebook


In [14]:
input_dataset = "s3://sagemaker-us-west-2-****/intel.zip"

In [ ]:
pytorch_processor.run(
    code='preprocess.py',
    source_dir='scripts',
    inputs=[
        ProcessingInput(
            input_name='data',
            source=input_dataset,
            destination='/opt/ml/processing/input'
        )
    ]
)